In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.io import write_json, read_json
from dash import dash_table
from dash.dash_table.Format import Format, Scheme
import pickle
import math
import os
import pandas as pd

In [ ]:
team_color = {
    'LG':['#C30452', '#000000'],
    'KT':['#000000', '#EB1C24'],
    '두산':['#FFFFFF', '#131230'],
    'KIA':['#EA0029', '#06141F'],
    '삼성':['#C0C0C0', '#074CA1']
}

In [ ]:
def log5(team_ratio: float, opponent_ratio: float) -> float:
    return team_ratio * (1-opponent_ratio) / (team_ratio * (1-opponent_ratio) + opponent_ratio * (1-team_ratio))

def postseason_ratio(winto, team_ratio, team_win = 0, opponent_win = 0):
    if team_win >= winto:
        return [(winto, opponent_win, 1.0)]
    elif opponent_win >= winto:
        return [(team_win, winto, 1.0)]
    opponent_ratio = 1 - team_ratio
    team_rest = winto - team_win
    opponent_rest = winto - opponent_win
    return [(winto, opponent_win + x, team_ratio ** team_rest * opponent_ratio ** x * math.comb(team_rest + x - 1, team_rest - 1)) for x in range(opponent_rest)] + [(team_win + x , winto, opponent_ratio ** opponent_rest * team_ratio ** x * math.comb(opponent_rest + x - 1, opponent_rest - 1)) for x in range(team_rest)]

In [ ]:
wc_fig = go.Figure(layout = go.Layout(hovermode = 'x'))
spo_fig = go.Figure(layout = go.Layout(hovermode = 'x'))
po_fig = go.Figure(layout = go.Layout(hovermode = 'x'))
ks_fig = go.Figure(layout = go.Layout(hovermode = 'x'))
wc_fig.update_xaxes(title_text = '게임 수', range = [0, 2], fixedrange = True, dtick = 1)
spo_fig.update_xaxes(title_text = '게임 수', range = [0, 5], fixedrange = True, dtick = 1)
po_fig.update_xaxes(title_text = '게임 수', range = [0, 5], fixedrange = True, dtick = 1)
ks_fig.update_xaxes(title_text = '게임 수', range = [0, 7], fixedrange = True, dtick = 1)
wc_fig.update_yaxes(range = [0, 1], fixedrange = True, tickformat = ',.3%')
spo_fig.update_yaxes(range = [0, 1], fixedrange = True, tickformat = ',.3%')
po_fig.update_yaxes(range = [0, 1], fixedrange = True, tickformat = ',.3%')
ks_fig.update_yaxes(range = [0, 1], fixedrange = True, tickformat = ',.3%')
wc_fig.update_layout(title_text = '진출 확률', margin_l=10, margin_r=10, margin_b=10, margin_t=50, plot_bgcolor='#D9F2D0', paper_bgcolor="#DFDFDF")
spo_fig.update_layout(title_text = '진출 확률', margin_l=10, margin_r=10, margin_b=10, margin_t=50, plot_bgcolor='#D9F2D0', paper_bgcolor="#DFDFDF")
po_fig.update_layout(title_text = '진출 확률', margin_l=10, margin_r=10, margin_b=10, margin_t=50, plot_bgcolor='#D9F2D0', paper_bgcolor="#DFDFDF")
ks_fig.update_layout(title_text = '진출 확률', margin_l=10, margin_r=10, margin_b=10, margin_t=50, plot_bgcolor='#D9F2D0', paper_bgcolor="#DFDFDF")

In [ ]:
ds_gamewin = log5(0.521, 0.507)
ds_win = [sum([a[2] for a in postseason_ratio(2, ds_gamewin, x, y) if a[0] == 2]) 
    for x, y in [(1,0), (1,1), (1,2)]]
kt_win = list(map(lambda x: 1 - x, ds_win))

In [ ]:
wc_fig.add_traces([
    # 두산 정사
    go.Scatter(
        name = '두산', 
        x = [0, 1, 2],
        y = ds_win,
        text = ['1승 0패', '1승 1패', '1승 2패'],
        mode = 'lines', line = {'color': team_color['두산'][0], 'width' : 3, 'dash': 'solid'}, marker = {'color': team_color['두산'][1], 'size': 3},
        legendgroup = '두산',
        visible = 'legendonly'
    ),
    # kt 정사
    go.Scatter(
        name = 'kt', 
        x = [0, 1, 2],
        y = kt_win,
        text = ['0승 1패', '1승 1패', '2승 1패'],
        mode = 'lines', line = {'color': team_color['KT'][0], 'width' : 3, 'dash': 'solid'}, marker = {'color': team_color['KT'][1], 'size': 3},
        legendgroup = 'kt'
    ),
    # 두산 1차전 승
    go.Scatter(
        x = [0, 1],
        y = [ds_win[0], 1.0],
        mode = 'lines', line = {'color': team_color['두산'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['두산'][1], 'size': 3},
        legendgroup = '두산',
        hoverinfo='skip',
        showlegend= False,
        visible = 'legendonly'
    ),
    # 두산 2차전 승
    go.Scatter(
        x = [1, 2],
        y = [ds_win[1], 1.0],
        mode = 'lines', line = {'color': team_color['두산'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['두산'][1], 'size': 3},
        legendgroup = '두산',
        hoverinfo='skip',
        showlegend= False,
        visible = 'legendonly'
    ),
    # kt 1차전 패
    go.Scatter(
        name = 'kt', 
        x = [0, 1],
        y = [kt_win[0], 0.0],
        mode = 'lines', line = {'color': team_color['KT'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['KT'][1], 'size': 3},
        legendgroup = 'kt',
        hoverinfo='skip',
        showlegend= False
    ),
    # kt 2차전 패
    go.Scatter(
        name = 'kt', 
        x = [1, 2],
        y = [kt_win[1], 0.0],
        mode = 'lines', line = {'color': team_color['KT'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['KT'][1], 'size': 3},
        legendgroup = 'kt',
        hoverinfo='skip',
        showlegend= False
    )
])

In [ ]:
lg_gamewin = log5(0.535, 0.507)
lg_win = [sum([a[2] for a in postseason_ratio(3, lg_gamewin, x, y) if a[0] == 3]) 
    for x, y in [(0,0), (0,1), (1,1)]]
kt_win = list(map(lambda x: 1 - x, lg_win))

In [ ]:
spo_fig.add_traces([
    # LG 정사
    go.Scatter(
        name = 'LG', 
        x = [0, 1, 2],
        y = lg_win,
        text = ['0승 0패', '0승 1패', '1승 1패'],
        mode = 'lines', line = {'color': team_color['LG'][0], 'width' : 3, 'dash': 'solid'}, marker = {'color': team_color['LG'][1], 'size': 3},
        legendgroup = 'LG'
    ),
    # kt 정사
    go.Scatter(
        name = 'kt', 
        x = [0, 1, 2],
        y = kt_win,
        text = ['0승 0패', '1승 0패', '1승 1패'],
        mode = 'lines', line = {'color': team_color['KT'][0], 'width' : 3, 'dash': 'solid'}, marker = {'color': team_color['KT'][1], 'size': 3},
        legendgroup = 'kt',
        visible = 'legendonly'
    ),
    # LG 1차전 승
    go.Scatter(
        x = [0, 1],
        y = [lg_win[0], sum([x[2] for x in postseason_ratio(3, lg_gamewin, 1, 0) if x[0] == 3])],
        mode = 'lines', line = {'color': team_color['LG'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['LG'][1], 'size': 3},
        legendgroup = 'LG',
        hoverinfo='skip',
        showlegend= False
    ),
    # LG 2차전 패
    go.Scatter(
        x = [1, 2],
        y = [lg_win[1], sum([x[2] for x in postseason_ratio(3, lg_gamewin, 0, 2) if x[0] == 3])],
        mode = 'lines', line = {'color': team_color['LG'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['LG'][1], 'size': 3},
        legendgroup = 'LG',
        hoverinfo='skip',
        showlegend= False
    ),
    # kt 1차전 패
    go.Scatter(
        name = 'kt', 
        x = [0, 1],
        y = [kt_win[0], sum([x[2] for x in postseason_ratio(3, 1 - lg_gamewin, 0, 1) if x[0] == 3])],
        mode = 'lines', line = {'color': team_color['KT'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['KT'][1], 'size': 3},
        legendgroup = 'kt',
        hoverinfo='skip',
        showlegend= False,
        visible = 'legendonly'
    ),
    # kt 2차전 승
    go.Scatter(
        name = 'kt', 
        x = [1, 2],
        y = [kt_win[1], sum([x[2] for x in postseason_ratio(3, 1 - lg_gamewin, 2, 0) if x[0] == 3])],
        mode = 'lines', line = {'color': team_color['KT'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['KT'][1], 'size': 3},
        legendgroup = 'kt',
        hoverinfo='skip',
        showlegend= False,
        visible = 'legendonly'
    ),
    # LG 미래시
    go.Scatter(
        x = [2, 3],
        y = [lg_win[-1], sum([x[2] for x in postseason_ratio(3, lg_gamewin, 2, 1) if x[0] == 3])],
        mode = 'lines', line = {'color': team_color['LG'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['LG'][1], 'size': 3},
        legendgroup = 'LG',
        hoverinfo='skip',
        showlegend= False
    ),
    go.Scatter(
        x = [2, 3],
        y = [lg_win[-1], sum([x[2] for x in postseason_ratio(3, lg_gamewin, 1, 2) if x[0] == 3])],
        mode = 'lines', line = {'color': team_color['LG'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['LG'][1], 'size': 3},
        legendgroup = 'LG',
        hoverinfo='skip',
        showlegend= False
    ),
    # kt 미래시
    go.Scatter(
        name = 'kt', 
        x = [2, 3],
        y = [kt_win[-1], sum([x[2] for x in postseason_ratio(3, 1 - lg_gamewin, 2, 1) if x[0] == 3])],
        mode = 'lines', line = {'color': team_color['KT'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['KT'][1], 'size': 3},
        legendgroup = 'kt',
        hoverinfo='skip',
        showlegend= False,
        visible = 'legendonly'
    ),
    go.Scatter(
        name = 'kt', 
        x = [2, 3],
        y = [kt_win[-1], sum([x[2] for x in postseason_ratio(3, 1 - lg_gamewin, 1, 2) if x[0] == 3])],
        mode = 'lines', line = {'color': team_color['KT'][0], 'width' : 3, 'dash': 'dash'}, marker = {'color': team_color['KT'][1], 'size': 3},
        legendgroup = 'kt',
        hoverinfo='skip',
        showlegend= False,
        visible = 'legendonly'
    )
])

In [ ]:
wc_probability = dash_table.DataTable([
    {
        '(두산-kt)': '확률',
        '2-0': '51.401%',
        '2-1': '24.980%',
        '1-2': '23.619%',
        '두산 진출': '76.381%',
        'kt 진출': '23.619%'
    }
],
[{'name': i, 'id': i} for i in ['(두산-kt)', '2-0', '2-1', '두산 진출', '1-2', 'kt 진출']],
style_cell_conditional=[
    {'if': {'column_id': ['1-2', 'kt 진출']}, 'fontWeight': 'bold', 'backgroundColor': '#BEF5CE'},
    {'if': {'column_id': ['두산 진출', 'kt 진출']}, 'border-left': '2px solid black' , 'border-right': '2px solid black'}],
style_header = {'text-align': 'center', 'fontWeight': 'bold'},
style_data = {'text-align': 'center', 'padding': '3px'},
style_table={'margin-left': 'auto', 'margin-right': 'auto', 'margin-top': '10px', 'margin-bottom': '10px', 'width': '100%', 'max-width': '600px'})

In [ ]:
spo_probability = dash_table.DataTable([
    {
        '(LG-kt)': '시리즈 초기',
        '3-0': '14.722%',
        '3-1': '20.845%',
        '3-2': '19.677%',
        '2-3': '17.588%',
        '1-3': '16.654%',
        '0-3': '10.514%',
        'LG 진출': '55.244%',
        'kt 진출': '44.756%'
    },
    {
        '(LG-kt)': '현재',
        '3-0': '0%',
        '3-1': '27.881%',
        '3-2': '26.318%',
        '2-3': '23.524%',
        '1-3': '22.276%',
        '0-3': '0%',
        'LG 진출': '54.199%',
        'kt 진출': '45.801%'
    }],
    [{'name': i, 'id': i} for i in ['(LG-kt)', '3-0', '3-1', '3-2', 'LG 진출', '0-3', '1-3', '2-3', 'kt 진출']],
    style_cell_conditional=[
        {'if': {'column_id': ['3-0', '0-3']}, 'backgroundColor': '#C0C0C0'},
        {'if': {'column_id': ['LG 진출', 'kt 진출']}, 'border-left': '2px solid black' , 'border-right': '2px solid black'}],
    style_header = {'text-align': 'center', 'fontWeight': 'bold'},
    style_data = {'text-align': 'center', 'padding': '3px'},
    style_table={'margin-left': 'auto', 'margin-right': 'auto', 'margin-top': '10px', 'margin-bottom': '10px', 'width': '100%', 'max-width': '800px'}
)

In [ ]:
po_probability = dash_table.DataTable([
    {
        '(삼성-???)': 'LG',
        '3-0': '13.587%',
        '3-1': '19.806',
        '3-2': '19.248%',
        '2-3': '18.192%',
        '1-3': '17.694%',
        '0-3': '11.472%',
        '삼성 진출': '52.641%',
        '상대팀 진출': '47.359%'
    },
    {
        '(삼성-???)': 'kt',
        '3-0': '15.927%',
        '3-1': '21.881%',
        '3-2': '20.041%',
        '2-3': '16.931%',
        '1-3': '15.617%',
        '0-3': '9.604%',
        '삼성 진출': '57.849%',
        '상대팀 진출': '42.151%'
    }],
    [{'name': i, 'id': i} for i in ['(삼성-???)', '3-0', '3-1', '3-2', '삼성 진출', '0-3', '1-3', '2-3', '상대팀 진출']],
    style_cell_conditional=[
        {'if': {'column_id': ['삼성 진출', '상대팀 진출']}, 'border-left': '2px solid black' , 'border-right': '2px solid black'}],
    style_header = {'text-align': 'center', 'fontWeight': 'bold'},
    style_data = {'text-align': 'center', 'padding': '3px'},
    style_table={'margin-left': 'auto', 'margin-right': 'auto', 'margin-top': '10px', 'margin-bottom': '10px', 'width': '100%', 'max-width': '800px'}
)

In [ ]:
ks_probability = dash_table.DataTable([
    {
        '(KIA-???)': '삼성',
        '4-0': '10.222%',
        '4-1': '17.769%',
        '4-2': '19.304%',
        '4-3': '16.777%',
        '0-4': '3.565%',
        '1-4': '8.065%',
        '2-4': '11.401%',
        '3-4': '12.893%',
        'KIA 우승': '64.072%',
        '상대팀 우승': '35.928%'
    },
    {
        '(KIA-???)': 'LG',
        '4-0': '11.258%',
        '4-1': '18.947%',
        '4-2': '19.930%',
        '4-3': '16.771%',
        '0-4': '3.133%',
        '1-4': '7.261%',
        '2-4': '10.515%',
        '3-4': '12.181%',
        'KIA 우승': '66.909%',
        '상대팀 우승': '33.091%'
    },
    {
        '(KIA-???)': 'kt',
        '4-0': '13.516%',
        '4-1': '21.283%',
        '4-2': '20.946%',
        '4-3': '16.491%',
        '0-4': '2.401%',
        '1-4': '5.824%',
        '2-4': '8.829%',
        '3-4': '10.707%',
        'KIA 우승': '72.281%',
        '상대팀 우승': '27.719%'
    }],
    [{'name': i, 'id': i} for i in ['(KIA-???)', '4-0', '4-1', '4-2', '4-3', 'KIA 우승', '0-4', '1-4', '2-4', '3-4', '상대팀 우승']],
    style_cell_conditional=[
        {'if': {'column_id': ['KIA 우승', '상대팀 우승']}, 'border-left': '2px solid black' , 'border-right': '2px solid black'}],
    style_header = {'text-align': 'center', 'fontWeight': 'bold'},
    style_data = {'text-align': 'center', 'padding': '3px'},
    style_table={'margin-left': 'auto', 'margin-right': 'auto', 'margin-top': '10px', 'margin-bottom': '10px', 'width': '100%', 'max-width': '800px'}
)

In [ ]:
write_json(wc_fig, file = 'fig/wc_fig.json', engine = 'json')
write_json(spo_fig, file = 'fig/spo_fig.json', engine = 'json')
write_json(po_fig, file = 'fig/po_fig.json', engine = 'json')
write_json(ks_fig, file = 'fig/ks_fig.json', engine = 'json')

In [ ]:
with open('fig/wc_table.pkl', 'wb') as fw:
    pickle.dump(obj = wc_probability, file = fw)
with open('fig/spo_table.pkl', 'wb') as fw:
    pickle.dump(obj = spo_probability, file = fw)
with open('fig/po_table.pkl', 'wb') as fw:
    pickle.dump(obj = po_probability, file = fw)
with open('fig/ks_table.pkl', 'wb') as fw:
    pickle.dump(obj = ks_probability, file = fw)